In [2]:
# -*- coding: utf-8 -*-
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np # 数据处理的库 numpy
import argparse
import imutils
import time
import dlib
import cv2
import math
import time
from threading import Thread
 
# 世界坐标系(UVW)：填写3D参考点，该模型参考http://aifi.isr.uc.pt/Downloads/OpenGL/glAnthropometric3DModel.cpp
object_pts = np.float32([[6.825897, 6.760612, 4.402142],  #33左眉左上角
                         [1.330353, 7.122144, 6.903745],  #29左眉右角
                         [-1.330353, 7.122144, 6.903745], #34右眉左角
                         [-6.825897, 6.760612, 4.402142], #38右眉右上角
                         [5.311432, 5.485328, 3.987654],  #13左眼左上角
                         [1.789930, 5.393625, 4.413414],  #17左眼右上角
                         [-1.789930, 5.393625, 4.413414], #25右眼左上角
                         [-5.311432, 5.485328, 3.987654], #21右眼右上角
                         [2.005628, 1.409845, 6.165652],  #55鼻子左上角
                         [-2.005628, 1.409845, 6.165652], #49鼻子右上角
                         [2.774015, -2.080775, 5.048531], #43嘴左上角
                         [-2.774015, -2.080775, 5.048531],#39嘴右上角
                         [0.000000, -3.116408, 6.097667], #45嘴中央下角
                         [0.000000, -7.415691, 4.070434]])#6下巴角

# 相机坐标系(XYZ)：添加相机内参
K = [6.5308391993466671e+002, 0.0, 3.1950000000000000e+002,
     0.0, 6.5308391993466671e+002, 2.3950000000000000e+002,
     0.0, 0.0, 1.0]# 等价于矩阵[fx, 0, cx; 0, fy, cy; 0, 0, 1]
# 图像中心坐标系(uv)：相机畸变参数[k1, k2, p1, p2, k3]
D = [7.0834633684407095e-002, 6.9140193737175351e-002, 0.0, 0.0, -1.3073460323689292e+000]

# 像素坐标系(xy)：填写凸轮的本征和畸变系数
cam_matrix = np.array(K).reshape(3, 3).astype(np.float32)
dist_coeffs = np.array(D).reshape(5, 1).astype(np.float32)



# 重新投影3D点的世界坐标轴以验证结果姿势
reprojectsrc = np.float32([[10.0, 10.0, 10.0],
                           [10.0, 10.0, -10.0],
                           [10.0, -10.0, -10.0],
                           [10.0, -10.0, 10.0],
                           [-10.0, 10.0, 10.0],
                           [-10.0, 10.0, -10.0],
                           [-10.0, -10.0, -10.0],
                           [-10.0, -10.0, 10.0]])
# 绘制正方体12轴
line_pairs = [[0, 1], [1, 2], [2, 3], [3, 0],
              [4, 5], [5, 6], [6, 7], [7, 4],
              [0, 4], [1, 5], [2, 6], [3, 7]]

def get_head_pose(shape):# 头部姿态估计
    # （像素坐标集合）填写2D参考点，注释遵循https://ibug.doc.ic.ac.uk/resources/300-W/
    # 17左眉左上角/21左眉右角/22右眉左上角/26右眉右上角/36左眼左上角/39左眼右上角/42右眼左上角/
    # 45右眼右上角/31鼻子左上角/35鼻子右上角/48左上角/54嘴右上角/57嘴中央下角/8下巴角
    image_pts = np.float32([shape[17], shape[21], shape[22], shape[26], shape[36],
                            shape[39], shape[42], shape[45], shape[31], shape[35],
                            shape[48], shape[54], shape[57], shape[8]])
    # solvePnP计算姿势——求解旋转和平移矩阵：
    # rotation_vec表示旋转矩阵，translation_vec表示平移矩阵，cam_matrix与K矩阵对应，dist_coeffs与D矩阵对应。
    _, rotation_vec, translation_vec = cv2.solvePnP(object_pts, image_pts, cam_matrix, dist_coeffs)
    # projectPoints重新投影误差：原2d点和重投影2d点的距离（输入3d点、相机内参、相机畸变、r、t，输出重投影2d点）
    reprojectdst, _ = cv2.projectPoints(reprojectsrc, rotation_vec, translation_vec, cam_matrix,dist_coeffs)
    reprojectdst = tuple(map(tuple, reprojectdst.reshape(8, 2)))# 以8行2列显示

    # 计算欧拉角calc euler angle
    # 参考https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#decomposeprojectionmatrix
    rotation_mat, _ = cv2.Rodrigues(rotation_vec)#罗德里格斯公式（将旋转矩阵转换为旋转向量）
    pose_mat = cv2.hconcat((rotation_mat, translation_vec))# 水平拼接，vconcat垂直拼接
    # decomposeProjectionMatrix将投影矩阵分解为旋转矩阵和相机矩阵
    _, _, _, _, _, _, euler_angle = cv2.decomposeProjectionMatrix(pose_mat)
    
    pitch, yaw, roll = [math.radians(_) for _ in euler_angle]
 
 
    pitch = math.degrees(math.asin(math.sin(pitch)))
    roll = -math.degrees(math.asin(math.sin(roll)))
    yaw = math.degrees(math.asin(math.sin(yaw)))
    print('pitch:{}, yaw:{}, roll:{}'.format(pitch, yaw, roll))

    return reprojectdst, euler_angle# 投影误差，欧拉角

def eye_aspect_ratio(eye):
    # 垂直眼标志（X，Y）坐标
    A = dist.euclidean(eye[1], eye[5])# 计算两个集合之间的欧式距离
    B = dist.euclidean(eye[2], eye[4])
    # 计算水平之间的欧几里得距离
    # 水平眼标志（X，Y）坐标
    C = dist.euclidean(eye[0], eye[3])
    # 眼睛长宽比的计算
    ear = (A + B) / (2.0 * C)
    # 返回眼睛的长宽比
    return ear
 
def mouth_aspect_ratio(mouth):# 嘴部
    A = np.linalg.norm(mouth[2] - mouth[9])  # 51, 59
    B = np.linalg.norm(mouth[4] - mouth[7])  # 53, 57
    C = np.linalg.norm(mouth[0] - mouth[6])  # 49, 55
    mar = (A + B) / (2.0 * C)
    return mar

# 定义常数
# 眼睛长宽比
# 闪烁阈值
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3
# 打哈欠长宽比
# 闪烁阈值
MAR_THRESH = 0.5
MOUTH_AR_CONSEC_FRAMES = 3
# 瞌睡点头
HAR_THRESH = 3.2
NOD_AR_CONSEC_FRAMES = 7
# 初始化帧计数器和眨眼总数
COUNTER = 0
TOTAL = 0
# 初始化帧计数器和打哈欠总数
mCOUNTER = 0
mTOTAL = 0
# 初始化帧计数器和点头总数
hCOUNTER = 0
hTOTAL = 0

# 初始化DLIB的人脸检测器（HOG），然后创建面部标志物预测
print("[INFO] loading facial landmark predictor...")
# 第一步：使用dlib.get_frontal_face_detector() 获得脸部位置检测器
detector = dlib.get_frontal_face_detector()
# 第二步：使用dlib.shape_predictor获得脸部特征位置检测器
predictor = dlib.shape_predictor('./data/shape_predictor_68_face_landmarks.dat')
 
# 第三步：分别获取左右眼面部标志的索引
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# 第四步：打开cv2 本地摄像头
cap = cv2.VideoCapture(0)
 
# 从视频流循环帧
while True:
    # 第五步：进行循环，读取图片，并对图片做维度扩大，并进灰度化
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=720)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 第六步：使用detector(gray, 0) 进行脸部位置检测
    rects = detector(gray, 0)
    
    # 第七步：循环脸部位置信息，使用predictor(gray, rect)获得脸部特征位置的信息
    for rect in rects:
        shape = predictor(gray, rect)
        
        # 第八步：将脸部特征信息转换为数组array的格式
        shape = face_utils.shape_to_np(shape)
        
        # 第九步：提取左眼和右眼坐标
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        # 嘴巴坐标
        mouth = shape[mStart:mEnd]        
        
        # 第十步：构造函数计算左右眼的EAR值，使用平均值作为最终的EAR
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        # 打哈欠
        mar = mouth_aspect_ratio(mouth)
 
        # 第十一步：使用cv2.convexHull获得凸包位置，使用drawContours画出轮廓位置进行画图操作
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
 
        # 第十二步：进行画图操作，用矩形框标注人脸
        left = rect.left()
        top = rect.top()
        right = rect.right()
        bottom = rect.bottom()
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 1)    
 
        '''
            分别计算左眼和右眼的评分求平均作为最终的评分，如果小于阈值，则加1，如果连续3次都小于阈值，则表示进行了一次眨眼活动
        '''
        # 第十三步：循环，满足条件的，眨眼次数+1
        if ear < EYE_AR_THRESH:# 眼睛长宽比：0.2
            COUNTER += 1
           
        else:
            # 如果连续3次都小于阈值，则表示进行了一次眨眼活动
            if COUNTER >= EYE_AR_CONSEC_FRAMES:# 阈值：3
                TOTAL += 1
            # 重置眼帧计数器
            COUNTER = 0
            
        # 第十四步：进行画图操作，同时使用cv2.putText将眨眼次数进行显示
        cv2.putText(frame, "Faces: {}".format(len(rects)), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)     
        cv2.putText(frame, "COUNTER: {}".format(COUNTER), (150, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (450, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        
        '''
            计算张嘴评分，如果小于阈值，则加1，如果连续3次都小于阈值，则表示打了一次哈欠，同一次哈欠大约在3帧
        '''
        # 同理，判断是否打哈欠    
        if mar > MAR_THRESH:# 张嘴阈值0.5
            mCOUNTER += 1
            cv2.putText(frame, "Yawning!", (10, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # 如果连续3次都小于阈值，则表示打了一次哈欠
            if mCOUNTER >= MOUTH_AR_CONSEC_FRAMES:# 阈值：3
                mTOTAL += 1
            # 重置嘴帧计数器
            mCOUNTER = 0
        cv2.putText(frame, "COUNTER: {}".format(mCOUNTER), (150, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
        cv2.putText(frame, "MAR: {:.2f}".format(mar), (300, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Yawning: {}".format(mTOTAL), (450, 60),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        """
        瞌睡点头
        """
        # 第十五步：获取头部姿态
        reprojectdst, euler_angle = get_head_pose(shape)
        
        har = euler_angle[0, 0]# 取pitch旋转角度
        if har > HAR_THRESH:# 点头阈值0.3
            hCOUNTER += 1
        else:
            # 如果连续3次都小于阈值，则表示瞌睡点头一次
            if hCOUNTER >= NOD_AR_CONSEC_FRAMES:# 阈值：3
                hTOTAL += 1
            # 重置点头帧计数器
            hCOUNTER = 0
        
        # 绘制正方体12轴
        for start, end in line_pairs:
            cv2.line(frame, reprojectdst[start], reprojectdst[end], (0, 0, 255))
        # 显示角度结果
        cv2.putText(frame, "X: " + "{:7.2f}".format(euler_angle[0, 0]), (10, 90), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), thickness=2)# GREEN
        cv2.putText(frame, "Y: " + "{:7.2f}".format(euler_angle[1, 0]), (150, 90), cv2.FONT_HERSHEY_SIMPLEX,0.75, (255, 0, 0), thickness=2)# BLUE
        cv2.putText(frame, "Z: " + "{:7.2f}".format(euler_angle[2, 0]), (300, 90), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), thickness=2)# RED    
        cv2.putText(frame, "Nod: {}".format(hTOTAL), (450, 90),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
        
            
        # 第十六步：进行画图操作，68个特征点标识
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
    
        print('嘴巴实时长宽比:{:.2f} '.format(mar)+"\t是否张嘴："+str([False,True][mar > MAR_THRESH]))
        print('眼睛实时长宽比:{:.2f} '.format(ear)+"\t是否眨眼："+str([False,True][COUNTER>=1]))
        
        # 确定疲劳提示:眨眼50次，打哈欠15次，瞌睡点头15次
        if TOTAL >= 50 or mTOTAL>=15 or hTOTAL>=15:
            cv2.putText(frame, "SLEEP!!!", (100, 200),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
            
        # 按q退出
        cv2.putText(frame, "Press 'q': Quit", (20, 500),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (84, 255, 159), 2)
        # 窗口显示 show with opencv
        cv2.imshow("Frame", frame)
        
        # if the `q` key was pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# 释放摄像头 release camera
cap.release()
# do a bit of cleanup
cv2.destroyAllWindows()

[INFO] loading facial landmark predictor...
pitch:8.566820230590572, yaw:6.088815664063385, roll:-2.543574876704907
嘴巴实时长宽比:0.29 	是否张嘴：False
眼睛实时长宽比:0.16 	是否眨眼：True
pitch:6.766791505065674, yaw:4.020170127156583, roll:-2.2416717024447563
嘴巴实时长宽比:0.31 	是否张嘴：False
眼睛实时长宽比:0.16 	是否眨眼：True
pitch:5.9509241788033345, yaw:3.2851080947113642, roll:-1.9531447257719963
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:5.715966782494353, yaw:3.0541763351969156, roll:-1.6291457316524633
嘴巴实时长宽比:0.29 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True


<ipython-input-2-d4e569311706>:245: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.line(frame, reprojectdst[start], reprojectdst[end], (0, 0, 255))
<ipython-input-2-d4e569311706>:257: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  print('嘴巴实时长宽比:{:.2f} '.format(mar)+"\t是否张嘴："+str([False,True][mar > MAR_THRESH]))


pitch:5.374129853032765, yaw:3.053989778730445, roll:-1.4798620383819943
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:5.4966411300456866, yaw:4.2551749412301625, roll:-1.506728078203679
嘴巴实时长宽比:0.31 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:4.392220617801088, yaw:2.4493096868231636, roll:-1.453784959451885
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:3.378194425929464, yaw:2.2401805491574245, roll:-1.6488558967151
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:3.385193830924985, yaw:2.1347379453395425, roll:-1.846040383090111
嘴巴实时长宽比:0.31 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：False
pitch:3.308067456634982, yaw:2.3126101997059245, roll:-1.9540283303124981
嘴巴实时长宽比:0.29 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:3.040668066087266, yaw:3.677856413989968, roll:-1.7992370250066059
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.19 	是否眨眼：True
pitch:1.591164742889978, yaw:2.8503539366044564, roll:-1.8857549112843648
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:1.55596596624330

嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:1.7598534337867784, yaw:2.2547146857277878, roll:-1.480620033916221
嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.16 	是否眨眼：True
pitch:1.6003304909115326, yaw:2.4203989016403575, roll:-1.5598576904160708
嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:2.2098252900428004, yaw:2.4425456920817297, roll:-1.4761248188561693
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.14 	是否眨眼：True
pitch:2.1972923804175744, yaw:2.8642693724193005, roll:-1.5183942210885946
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:1.569995810717063, yaw:2.479198207520997, roll:-1.6942162953863102
嘴巴实时长宽比:0.36 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:1.938680341913714, yaw:2.7326145799713983, roll:-1.6512631931324844
嘴巴实时长宽比:0.36 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:2.6285586919358463, yaw:0.7049454738642531, roll:-1.9348917701229356
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.42 	是否眨眼：False
pitch:0.31409347310341534, yaw:1.9725964480062708, roll:-1.7434882541168963
嘴巴实时长宽比:0.

嘴巴实时长宽比:0.36 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:-22.36714685835467, yaw:1.9885732567071803, roll:2.9359727628178742
嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:-21.03531852409736, yaw:0.8849319317555048, roll:3.204293507393636
嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.15 	是否眨眼：True
pitch:-10.098197038361333, yaw:1.98290476102379, roll:3.484487567699109
嘴巴实时长宽比:0.36 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:-5.8185090712964325, yaw:3.8384059508833963, roll:3.6608889476995907
嘴巴实时长宽比:0.36 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:-2.56719385061735, yaw:6.706170020297838, roll:2.3139229369630474
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.12 	是否眨眼：True
pitch:11.561028086359888, yaw:4.6804934048530225, roll:1.9638440703906872
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.35 	是否眨眼：False
pitch:-7.684457166646219, yaw:11.20323651132609, roll:3.030357947206502
嘴巴实时长宽比:0.40 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:5.662056993034327, yaw:6.008124218372912, roll:1.6392411345552227
嘴巴实时长宽比:0.31 	是否张嘴：Fals

...

pitch:4.44652010521662, yaw:7.7540635090672145, roll:1.8405589516165866
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:4.796128240677428, yaw:7.935100862081612, roll:2.105474898502335
嘴巴实时长宽比:0.32 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：True
pitch:3.8472358185621127, yaw:8.249720721235114, roll:2.00425249682664
嘴巴实时长宽比:0.31 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:4.215738137915213, yaw:7.201896601169486, roll:1.5687736976598383
嘴巴实时长宽比:0.32 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:2.2834647872685303, yaw:7.5885561081281026, roll:1.1833123816331497
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:2.1064224065305517, yaw:8.407262995987827, roll:1.864209156936958
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.19 	是否眨眼：True
pitch:4.233873911383431, yaw:7.450845714306156, roll:1.6332579171594768
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：False
pitch:3.9784415548110146, yaw:7.31628286543036, roll:1.1097459525131386
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.22 	是否眨眼：False
pitch:2.7752396896173375, y

pitch:1.1740916585799452, yaw:7.315320958542343, roll:0.906376730022799
嘴巴实时长宽比:0.23 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:0.7776213702305568, yaw:7.146698080153638, roll:0.8502697610730239
嘴巴实时长宽比:0.22 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：True
pitch:2.0629807902481927, yaw:8.21535340048698, roll:0.7545181573445718
嘴巴实时长宽比:0.23 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:1.8880112852400424, yaw:7.564236189305114, roll:0.8438325521793603
嘴巴实时长宽比:0.24 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：True
pitch:2.7985967663311793, yaw:7.74068230109468, roll:0.2874318313935655
嘴巴实时长宽比:0.22 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：True
pitch:2.228760212258929, yaw:8.089233712877125, roll:0.5596656031813237
嘴巴实时长宽比:0.22 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：True
pitch:1.4969676096638727, yaw:7.660377055570667, roll:0.5951440762650597
嘴巴实时长宽比:0.23 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:1.6585959747495076, yaw:8.296316991068668, roll:0.5202498602108633
嘴巴实时长宽比:0.25 	是否张嘴：False
眼睛实时长宽比:0.19 	是否眨眼：True
pitch:2.774214303819503, ya

嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:1.8501788487639028, yaw:10.372704650507966, roll:-1.4112297125159314
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.16 	是否眨眼：True
pitch:2.0373270265516252, yaw:11.89712965028486, roll:-1.0764515647009016
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:2.420144919646789, yaw:15.257202749351167, roll:0.40004702028284456
嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.25 	是否眨眼：False
pitch:8.075028980024387, yaw:15.05370384949421, roll:-3.377568231255408
嘴巴实时长宽比:0.30 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:3.370452018383481, yaw:15.182947554684379, roll:-1.6384298452791377
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:4.566248502537495, yaw:13.341638339259507, roll:-2.8052965209234846
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.19 	是否眨眼：True
pitch:3.9470119766577696, yaw:14.070583316384194, roll:-2.227125223661724
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.21 	是否眨眼：False
pitch:3.8556243179678495, yaw:13.832420548781457, roll:-2.5308214307040164
嘴巴实时长宽比:0.33 

嘴巴实时长宽比:0.32 	是否张嘴：False
眼睛实时长宽比:0.19 	是否眨眼：True
pitch:2.3799088063846865, yaw:13.04004484922712, roll:-1.260085421146198
嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.20 	是否眨眼：False
pitch:2.6478419142865524, yaw:12.791628441426065, roll:-1.3303291920274967
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:3.1057703851346723, yaw:12.912452602618263, roll:-1.4133948285635247
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:3.9392009731240676, yaw:12.228129103470495, roll:-1.8299780415201996
嘴巴实时长宽比:0.35 	是否张嘴：False
眼睛实时长宽比:0.18 	是否眨眼：True
pitch:3.6702845229394647, yaw:11.99118878900197, roll:-1.3201105242704783
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:3.7898175067990465, yaw:10.74706584945489, roll:-1.3694623817918112
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.16 	是否眨眼：True
pitch:3.5417312506542986, yaw:11.124778064575816, roll:-1.123207921073035
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.17 	是否眨眼：True
pitch:3.4301852179025576, yaw:9.778151799780915, roll:-0.9302680358587171
嘴巴实时长宽比:0.33 

嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.24 	是否眨眼：False
pitch:3.2624514379017584, yaw:13.616660465756672, roll:-1.3816249771988265
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
pitch:3.4641040686245756, yaw:10.674173464066257, roll:-0.5364355069128526
嘴巴实时长宽比:0.32 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
pitch:3.7729022873224927, yaw:11.425310725790425, roll:-0.8462106637325539
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.26 	是否眨眼：False
pitch:3.8336303921096335, yaw:10.482281718967602, roll:-0.6129401483204152
嘴巴实时长宽比:0.33 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
pitch:3.1185153517605797, yaw:11.553436612999084, roll:-0.5418913815912713
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.30 	是否眨眼：False
pitch:4.139273541264235, yaw:10.992160644890758, roll:-0.6719872193561774
嘴巴实时长宽比:0.34 	是否张嘴：False
眼睛实时长宽比:0.27 	是否眨眼：False
pitch:2.951592328104112, yaw:10.527395332761937, roll:-0.9794212426064401
嘴巴实时长宽比:0.37 	是否张嘴：False
眼睛实时长宽比:0.32 	是否眨眼：False
pitch:1.9978000930620028, yaw:10.234172421624905, roll:-0.24413956698109376
嘴